# MOwNiT - laboratorium 7

## Autor: Tomasz Boroń

## Temat: Rozwiązywanie układów równań liniowych metodami iteracyjnymi

#### Importy

In [2]:
import numpy as np
from sys import exit
from time import time

#### Definicja układu równań

In [3]:
def A_matrix(n):
    i, j = np.ogrid[1:n+1, 1:n+1]
    return np.where(i == j, 9, 1/(abs(i-j)+2))

In [4]:
def permutation(n):
    values = np.array([-1, 1])
    return np.random.choice(values, n)

In [5]:
def calc_system(n):
    A = A_matrix(n)
    x = permutation(n)
    return A, x, A @ x

#### Generowanie jednego układu

In [6]:
n = 6
A, true_x, b = calc_system(n)
print("A:",A)
print("Real x:",true_x)
print("b:",b)
print()
x = np.array([2]*n)
true_x = true_x[:, np.newaxis]
x = x[:, np.newaxis]
b = b[:, np.newaxis]

A: [[9.         0.33333333 0.25       0.2        0.16666667 0.14285714]
 [0.33333333 9.         0.33333333 0.25       0.2        0.16666667]
 [0.25       0.33333333 9.         0.33333333 0.25       0.2       ]
 [0.2        0.25       0.33333333 9.         0.33333333 0.25      ]
 [0.16666667 0.2        0.25       0.33333333 9.         0.33333333]
 [0.14285714 0.16666667 0.2        0.25       0.33333333 9.        ]]
Real x: [-1 -1 -1  1 -1  1]
b: [-9.40714286 -9.45       -9.3         8.13333333 -8.95        8.40714286]



#### Zadanie 1

In [7]:
def Jacobi(A, b, x0, prec, stop):
    n = x0.size
    i, j = np.ogrid[0:n, 0:n]
    iteration = 1
    while 1:
        if iteration == 10000: # zwrócenie None jest celowe - w ten sposób przerywam całą resztę wywołań, którą szukałem układu o promieniu spektralnym ponad 1
            print(x0)
            return None, None
        components = np.where(i != j, A[i,j]*x0[j,0], 0)
        c = np.sum(components, axis = 1).reshape(-1, 1)
        x1 = (b-c)/A[i,i]
        if stop == 1:
            if np.linalg.norm(x1-x0) < prec:
                return x1, iteration
        elif stop == 2:
            if np.linalg.norm(A @ x1 - b) < prec:
                return x1, iteration
        else:
            print("Bad criterium number")
            return x0, iteration
        iteration += 1
        x0 = x1

In [8]:
def calc_result_prec(x, true_x):
    return np.max(abs(x-true_x))

In [9]:
result, i = Jacobi(A, b, x, 1e-4, 2)
print("Got ", result, " after ", i, "iterations")
print("Result precision: ", calc_result_prec(result, true_x))

Got  [[-1.00000216]
 [-1.00000246]
 [-1.0000026 ]
 [ 0.9999974 ]
 [-1.00000246]
 [ 0.99999784]]  after  7 iterations
Result precision:  2.603542481494081e-06


#### Zadanie 2

In [10]:
def find_iteration_matrix(A):
    n = A.shape[0]
    i, j = np.ogrid[0:n, 0:n]
    D = np.where(i == j, A[i,i], 0)
    M = np.eye(n) - (np.linalg.inv(D) @ A)
    return M

In [11]:
def spectral_radius_lib(M):
    eigenvalues = np.linalg.eig(M)[0]
    return np.max(abs(eigenvalues))

In [12]:
def spectral_radius(M):
    x0 = np.array([1]*M.shape[0])
    x0 = x0[:, np.newaxis]
    for i in range(200):
        x1 = (M @ x0)/(np.linalg.norm(M @ x0))
        x0 = x1
    return abs((x1.T@M@x1/(x1.T@x1)).item())

In [13]:
M = find_iteration_matrix(A)
print(spectral_radius_lib(M))
print(spectral_radius(M))

0.13961187741990214
0.1396118774199021


#### Zadanie 3

In [14]:
def sor(A, b, x0, prec, stop, omega):
    n = x0.size
    c = np.array([0.0]*n)
    c = c[:, np.newaxis]
    d = np.array([0.0]*n)
    d = d[:, np.newaxis]
    iteration = 1
    while 1:
        if iteration == 10000:
            return None, None
        x1 = np.array([0.0]*n)
        x1 = x1[:, np.newaxis]
        for i in range(n):
            for j in range(i):
                c[j,0] = A[i,j]*x1[j,0]
                d[j,0] = 0
            for j in range(i, n):
                c[j,0] = 0
                d[j,0] = A[i,j]*x0[j,0]
            x1[i,0] = x0[i,0] + omega*(b[i,0]-np.sum(c)-np.sum(d))/A[i,i]
        if stop == 1:
            if np.linalg.norm(x1-x0) < prec:
                return x1, iteration
        elif stop == 2:
            if np.linalg.norm(A @ x1 - b) < prec:
                return x1, iteration
        else:
            print("Bad criterium number")
            return x0, iteration
        iteration += 1
        x0 = x1

In [15]:
def sor_iteration_matrix(A, omega):
    n = A.shape[0]
    i, j = np.ogrid[0:n, 0:n]
    D = np.where(i==j, A[i, j], 0)
    L = np.where(i>j, A[i, j], 0)
    U = np.where(i<j, A[i, j], 0)
    return np.linalg.inv(D + omega * L) @ (D - omega * (D + U))

In [16]:
result, i = sor(A, b, x, 1e-4, 2, 0.6)
M = sor_iteration_matrix(A, 0.6)
rad = spectral_radius_lib(M)
print("SOR: Spectral radius ", rad)
print("SOR: Got ", result, " after ", i, "iterations")
print("Result precision: ", calc_result_prec(result, true_x))

SOR: Spectral radius  0.4159576470491603
SOR: Got  [[-0.99999972]
 [-0.99999897]
 [-0.99999639]
 [ 0.99999448]
 [-0.99999397]
 [ 0.99999953]]  after  14 iterations
Result precision:  6.026252758806194e-06


#### Testy całości

Metoda Jacobiego, precyzja 1e-3

In [ ]:
prec = 1e-3
sizes = [i for i in range(2,21)] + [i for i in range(30, 101, 10)]
print("Równań\titeracji\tdokładność_wyniku\tpromień_spektralny\tstop")
for elem in sizes:
    n = elem
    A, true_x, b = calc_system(n)
    x = np.random.uniform(-50, 50, n)
    true_x = true_x[:, np.newaxis]
    x = x[:, np.newaxis]
    b = b[:, np.newaxis]
    result, i = Jacobi(A, b, x, prec, 1)
    result_prec = calc_result_prec(result, true_x)
    M = find_iteration_matrix(A)
    print(n,"\t",i,"\t",result_prec,"\t",spectral_radius(M),"\t","1")
    result, i = Jacobi(A, b, x, prec, 2)
    result_prec = calc_result_prec(result, true_x)
    M = find_iteration_matrix(A)
    print(n,"\t",i,"\t",result_prec,"\t",spectral_radius(M),"\t","2")

Metoda Jacobiego, precyzja 1e-6

In [ ]:
prec = 1e-6
sizes = [i for i in range(2,21)] + [i for i in range(30, 101, 10)]
print("Równań\titeracji\tdokładność_wyniku\tpromień_spektralny\tstop")
for elem in sizes:
    n = elem
    A, true_x, b = calc_system(n)
    x = np.random.uniform(-50, 50, n)
    true_x = true_x[:, np.newaxis]
    x = x[:, np.newaxis]
    b = b[:, np.newaxis]
    result, i = Jacobi(A, b, x, prec, 1)
    result_prec = calc_result_prec(result, true_x)
    M = find_iteration_matrix(A)
    print(n,"\t",i,"\t",result_prec,"\t",spectral_radius(M),"\t","1")
    result, i = Jacobi(A, b, x, prec, 2)
    result_prec = calc_result_prec(result, true_x)
    M = find_iteration_matrix(A)
    print(n,"\t",i,"\t",result_prec,"\t",spectral_radius(M),"\t","2")

Metoda Jacobiego, precyzja 1e-9

In [ ]:
prec = 1e-9
sizes = [i for i in range(2,21)] + [i for i in range(30, 101, 10)]
print("Równań\titeracji\tdokładność_wyniku\tpromień_spektralny\tstop")
for elem in sizes:
    n = elem
    A, true_x, b = calc_system(n)
    x = np.random.uniform(-50, 50, n)
    true_x = true_x[:, np.newaxis]
    x = x[:, np.newaxis]
    b = b[:, np.newaxis]
    result, i = Jacobi(A, b, x, prec, 1)
    result_prec = calc_result_prec(result, true_x)
    M = find_iteration_matrix(A)
    print(n,"\t",i,"\t",result_prec,"\t",spectral_radius(M),"\t","1")
    result, i = Jacobi(A, b, x, prec, 2)
    result_prec = calc_result_prec(result, true_x)
    M = find_iteration_matrix(A)
    print(n,"\t",i,"\t",result_prec,"\t",spectral_radius(M),"\t","2")

Sprawdzenie jak wartość omegi wpływa na wyniki dla ustalonego układu

In [ ]:
prec = 1e-6
n = 20
print("Omega\titeracji\tdokładność_wyniku\tstop")
for omega in range(1, 20):
    A, true_x, b = calc_system(n)
    x = np.random.uniform(-50, 50, n)
    true_x = true_x[:, np.newaxis]
    x = x[:, np.newaxis]
    b = b[:, np.newaxis]
    result, i = sor(A, b, x, prec, 1, omega/10)
    result_prec = calc_result_prec(result, true_x)
    print(omega/10,"\t",i,"\t",result_prec,"\t","1")
    result, i = sor(A, b, x, prec, 2, omega/10)
    result_prec = calc_result_prec(result, true_x)
    print(omega/10,"\t",i,"\t",result_prec,"\t","2")

Metoda SOR, precyzja 1e-3, omega = 0.5

In [ ]:
prec = 1e-3
omega = 0.5
sizes = [i for i in range(2,21)] + [i for i in range(30, 101, 10)]
print("Równań\tstop\titeracji\tdokładność_wyniku")
for elem in sizes:
    n = elem
    A, true_x, b = calc_system(n)
    x = np.random.uniform(-50, 50, n)
    true_x = true_x[:, np.newaxis]
    x = x[:, np.newaxis]
    b = b[:, np.newaxis]
    result, i = sor(A, b, x, prec, 1, omega)
    result_prec = calc_result_prec(result, true_x)
    print(n, "\t", "1","\t",i,"\t",result_prec)
    result, i = sor(A, b, x, prec, 2, omega)
    result_prec = calc_result_prec(result, true_x)
    print(n, "\t", "2","\t",i,"\t",result_prec)

Metoda SOR, precyzja 1e-3, omega = 1.5

In [ ]:
prec = 1e-3
omega = 1.5
sizes = [i for i in range(2,21)] + [i for i in range(30, 101, 10)]
print("Równań\tstop\titeracji\tdokładność_wyniku")
for elem in sizes:
    n = elem
    A, true_x, b = calc_system(n)
    x = np.random.uniform(-50, 50, n)
    true_x = true_x[:, np.newaxis]
    x = x[:, np.newaxis]
    b = b[:, np.newaxis]
    result, i = sor(A, b, x, prec, 1, omega)
    result_prec = calc_result_prec(result, true_x)
    print(n, "\t", "1","\t",i,"\t",result_prec)
    result, i = sor(A, b, x, prec, 2, omega)
    result_prec = calc_result_prec(result, true_x)
    print(n, "\t", "2","\t",i,"\t",result_prec)

Metoda SOR, precyzja 1e-6, omega = 0.5

In [ ]:
prec = 1e-6
omega = 0.5
sizes = [i for i in range(2,21)] + [i for i in range(30, 101, 10)]
print("Równań\tstop\titeracji\tdokładność_wyniku")
for elem in sizes:
    n = elem
    A, true_x, b = calc_system(n)
    x = np.random.uniform(-50, 50, n)
    true_x = true_x[:, np.newaxis]
    x = x[:, np.newaxis]
    b = b[:, np.newaxis]
    result, i = sor(A, b, x, prec, 1, omega)
    result_prec = calc_result_prec(result, true_x)
    print(n, "\t", "1","\t",i,"\t",result_prec)
    result, i = sor(A, b, x, prec, 2, omega)
    result_prec = calc_result_prec(result, true_x)
    print(n, "\t", "2","\t",i,"\t",result_prec)

Metoda SOR, precyzja 1e-6, omega = 1.5

In [ ]:
prec = 1e-6
omega = 1.5
sizes = [i for i in range(2,21)] + [i for i in range(30, 101, 10)]
print("Równań\tstop\titeracji\tdokładność_wyniku")
for elem in sizes:
    n = elem
    A, true_x, b = calc_system(n)
    x = np.random.uniform(-50, 50, n)
    true_x = true_x[:, np.newaxis]
    x = x[:, np.newaxis]
    b = b[:, np.newaxis]
    result, i = sor(A, b, x, prec, 1, omega)
    result_prec = calc_result_prec(result, true_x)
    print(n, "\t", "1","\t",i,"\t",result_prec)
    result, i = sor(A, b, x, prec, 2, omega)
    result_prec = calc_result_prec(result, true_x)
    print(n, "\t", "2","\t",i,"\t",result_prec)

Metoda SOR, precyzja 1e-9, omega = 0.5

In [ ]:
prec = 1e-9
omega = 0.5
sizes = [i for i in range(2,21)] + [i for i in range(30, 101, 10)]
print("Równań\tstop\titeracji\tdokładność_wyniku")
for elem in sizes:
    n = elem
    A, true_x, b = calc_system(n)
    x = np.random.uniform(-50, 50, n)
    true_x = true_x[:, np.newaxis]
    x = x[:, np.newaxis]
    b = b[:, np.newaxis]
    result, i = sor(A, b, x, prec, 1, omega)
    result_prec = calc_result_prec(result, true_x)
    print(n, "\t", "1","\t",i,"\t",result_prec)
    result, i = sor(A, b, x, prec, 2, omega)
    result_prec = calc_result_prec(result, true_x)
    print(n, "\t", "2","\t",i,"\t",result_prec)

Metoda SOR, precyzja 1e-9, omega = 1.5

In [ ]:
prec = 1e-9
omega = 1.5
sizes = [i for i in range(2,21)] + [i for i in range(30, 101, 10)]
print("Równań\tstop\titeracji\tdokładność_wyniku")
for elem in sizes:
    n = elem
    A, true_x, b = calc_system(n)
    x = np.random.uniform(-50, 50, n)
    true_x = true_x[:, np.newaxis]
    x = x[:, np.newaxis]
    b = b[:, np.newaxis]
    result, i = sor(A, b, x, prec, 1, omega)
    result_prec = calc_result_prec(result, true_x)
    print(n, "\t", "1","\t",i,"\t",result_prec)
    result, i = sor(A, b, x, prec, 2, omega)
    result_prec = calc_result_prec(result, true_x)
    print(n, "\t", "2","\t",i,"\t",result_prec)

#### Czasy

In [ ]:
prec = 1e-9
sizes = [i for i in range(2,21)] + [i for i in range(30, 101, 10)]
print("Równań\tstop\titeracji\tczas")
for elem in sizes:
    n = elem
    A, true_x, b = calc_system(n)
    x = np.random.uniform(-50, 50, n)
    true_x = true_x[:, np.newaxis]
    x = x[:, np.newaxis]
    b = b[:, np.newaxis]
    start_time = time()
    result, i = Jacobi(A, b, x, prec, 1)
    end_time = time()
    print(n,"\t","1","\t",i,"\t",end_time-start_time)
    start_time = time()
    result, i = Jacobi(A, b, x, prec, 2)
    end_time = time()
    print(n,"\t","2","\t",i,"\t",end_time-start_time)

In [ ]:
prec = 1e-9
omega = 0.25
sizes = [i for i in range(2,21)] + [i for i in range(30, 101, 10)]
print("Równań\tstop\titeracji\tczas")
for elem in sizes:
    n = elem
    A, true_x, b = calc_system(n)
    x = np.random.uniform(-50, 50, n)
    true_x = true_x[:, np.newaxis]
    x = x[:, np.newaxis]
    b = b[:, np.newaxis]
    start_time = time()
    result, i = sor(A, b, x, prec, 1, omega)
    end_time = time()
    print(n,"\t","1","\t",i,"\t",end_time-start_time)
    start_time = time()
    result, i = sor(A, b, x, prec, 2, omega)
    end_time = time()
    print(n,"\t","2","\t",i,"\t",end_time-start_time)

#### Promień spektralny dla metody SOR

Dla ustalonego rozmiaru układu i różnego omega

In [17]:
prec = 1e-6
omegas = [x/10 for x in range(1,20)]
size = 50
A, true_x, b = calc_system(size)
x = np.random.uniform(-50, 50, size)
true_x = true_x[:, np.newaxis]
x = x[:, np.newaxis]
b = b[:, np.newaxis]
print("Omega\tpromień")
for omega in omegas:
    M = sor_iteration_matrix(A, omega)
    rad = spectral_radius_lib(M)
    print(omega, "\t", rad)

Omega	promień
0.1 	 0.904071985186227
0.2 	 0.8076958689609361
0.3 	 0.7108762885710865
0.4 	 0.6136137423618666
0.5 	 0.5159074562751714
0.6 	 0.4177549728428773
0.7 	 0.31915063104397695
0.8 	 0.22008099858090985
0.9 	 0.12050666994896707
1.0 	 0.1227527318241316
1.1 	 0.218114124369599
1.2 	 0.3121947713583073
1.3 	 0.4043705560403684
1.4 	 0.4946666560873927
1.5 	 0.5831576750418238
1.6 	 0.6699048707506394
1.7 	 0.7549464348222684
1.8 	 0.8383020694807254
1.9 	 0.9199828511761766


Dla ustalonego omega i różnych rozmiarów układu

In [18]:
prec = 1e-6
omega = 0.5
sizes = [i for i in range(10, 571, 10)]
print("Rozmiar\tpromień")
for size in sizes:
    A, true_x, b = calc_system(size)
    x = np.random.uniform(-50, 50, size)
    true_x = true_x[:, np.newaxis]
    x = x[:, np.newaxis]
    b = b[:, np.newaxis]
    M = sor_iteration_matrix(A, omega)
    rad = spectral_radius_lib(M)
    print(size, "\t", rad)

Rozmiar	promień
10 	 0.5150824376823735
20 	 0.5155387369196587
30 	 0.5157301946776965
40 	 0.5158377837171524
50 	 0.5159074562751714
60 	 0.5159565704672568
70 	 0.5159932135678239
80 	 0.5160216879134877
90 	 0.5160445035136255
100 	 0.5160632280577767
110 	 0.5160788935457178
120 	 0.5160922083652203
130 	 0.5161036756482866
140 	 0.5161136630305113
150 	 0.5161224456614102
160 	 0.5161302337455491
170 	 0.5161371907555433
180 	 0.5161434458166421
190 	 0.5161491023346085
200 	 0.516154244136734
210 	 0.5161589399256802
220 	 0.5161632465655859
230 	 0.5161672115432169
240 	 0.5161708748370527
250 	 0.5161742703545011
260 	 0.5161774270499464
270 	 0.5161803698027894
280 	 0.5161831201151427
290 	 0.5161856966696385
300 	 0.5161881157799825
310 	 0.516190391757288
320 	 0.5161925372104245
330 	 0.5161945632933027
340 	 0.5161964799104893
350 	 0.5161982958880347
360 	 0.5162000191180329
370 	 0.5162016566790658
380 	 0.5162032149397241
390 	 0.5162046996457463
400 	 0.516206115994